# **rectorch**: mult-vae model

## Preliminaries

### Dataset download
For the purposes of this tutorial we download the *movielens 20M* dataset. As the name suggests, this dataset contains roughly one million (5 stars) ratings about movies. For more details, please refer to the official web page https://grouplens.org/datasets/movielens/20m/.

In [1]:
%cd /content/
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip
!rm ml-20m.zip

/content
--2020-10-04 18:05:52--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  26.4MB/s    in 7.9s    

2020-10-04 18:06:00 (24.1 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [1]:
%cd /content/
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!rm ml-1m.zip

/content
--2020-10-08 10:14:07--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  14.9MB/s    in 0.4s    

2020-10-08 10:14:08 (14.9 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


### **rectorch** installation

NOTE: in this version of the tutorial we load the *dev* version from [github](https://github.com/makgyver/rectorch).

In [2]:
%cd /content/
!git clone -b dev https://github.com/makgyver/rectorch.git
%cd rectorch
!pip install -r requirements.txt

/content
Cloning into 'rectorch'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1705 (delta 17), reused 35 (delta 17), pack-reused 1650
Receiving objects: 100% (1705/1705), 3.22 MiB | 14.27 MiB/s, done.
Resolving deltas: 100% (1133/1133), done.
/content/rectorch


### Data loading and splitting

In [3]:
#ml20m
cfg_data = {
    "processing": {
        "data_path": "../ml-20m/ratings.csv",
        "threshold": 3.5,
        "separator": ",",
        "header": 0,
        "u_min": 5,
        "i_min": 0
    },
    "splitting": {
        "split_type": "vertical",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 10000,
        "test_size": 10000,
        "test_prop": 0.2
    }
}

In [3]:
#ml1m
cfg_data = {
    "processing": {
        "data_path": "../ml-1m/ratings.dat",
        "threshold": 3.5,
        "separator": "::",
        #"header": 0,
        "u_min": 5,
        "i_min": 0
    },
    "splitting": {
        "split_type": "vertical",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 200,
        "test_size": 200,
        "test_prop": 0.2
    }
}

In [5]:
#steam
cfg_data = {
    "processing": {
        "data_path": "../steam_sorted.csv",
        "threshold": 0,
        "separator": ",",
        #"header": 0,
        "u_min": 2,
        "i_min": 0
    },
    "splitting": {
        "split_type": "vertical",
        "sort_by": None,
        "seed": 98765,
        "shuffle": True,
        "valid_size": 1000,
        "test_size": 1000,
        "test_prop": 0.2
    }
}

In [6]:
from rectorch.data import DataProcessing
dataset = DataProcessing(cfg_data).process_and_split()
dataset

[10:17:32-081020]  Reading raw data file ../steam_sorted.csv.
[10:17:33-081020]  Applying filtering.
[10:17:33-081020]  Filtered 5700 ratings.
[10:17:33-081020]  Shuffling data.
[10:17:33-081020]  Calculating splits.
[10:17:33-081020]  Creating validation and test set.
[10:17:33-081020]  Skipped 186 ratings in validation set.
[10:17:33-081020]  Skipped 243 ratings in test set.
[10:17:33-081020]  Skipped 1 users in validation set.
[10:17:33-081020]  Skipped 3 users in test set.


Dataset(n_users=6693, n_items=4779, n_ratings=122671)

For more details about how to load, process and splitting the dataset, please refer to the tutorial [rectorch_data_tutorial.ipynb](https://colab.research.google.com/drive/1gKgMllkYlvvBqh7q6WmmSvtfAOTz7tFh#scrollTo=Cwi1HjgJ-T7Z).

### Sampler creation: rectorch.samplers.DataSampler



In [7]:
from rectorch.samplers import DataSampler
sampler = DataSampler(dataset, mode="train", batch_size=500)

The `mode` of a sampler indicates its current state, that is which part of the dataset is handling. In this case, the training set ("train") since we are going to train the models.

### Mult-VAE recommender

A random recommender is simply a system that recommends random items to users. The only useful parameter to initialize the model is the number of items.

In [8]:
from rectorch.models.nn.multvae import MultVAE
vae = MultVAE(dec_dims=[200,600,dataset.n_items],
              enc_dims=None,
              dropout=0.5,
              beta=.2,
              anneal_steps=100000,
              opt_conf=None,
              device="cuda",
              trainer=None)

[10:18:05-081020]  Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
[10:18:05-081020]  Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


### Training

In [9]:
vae.train(sampler, valid_metric="ndcg@100")

[10:18:30-081020]  | epoch 1 | 10/10 batches | ms/batch 39.51 | loss 144.45 |
[10:18:30-081020]  | epoch 1 | loss 144.4506 | total time: 0.40s |
[10:18:30-081020]  | epoch 1 | ndcg@100 0.240 (0.0088) |
[10:18:31-081020]  | epoch 2 | 10/10 batches | ms/batch 16.69 | loss 133.65 |
[10:18:31-081020]  | epoch 2 | loss 133.6478 | total time: 0.17s |
[10:18:31-081020]  | epoch 2 | ndcg@100 0.216 (0.0066) |
[10:18:31-081020]  | epoch 3 | 10/10 batches | ms/batch 16.82 | loss 132.08 |
[10:18:31-081020]  | epoch 3 | loss 132.0813 | total time: 0.17s |
[10:18:31-081020]  | epoch 3 | ndcg@100 0.250 (0.0087) |
[10:18:31-081020]  | epoch 4 | 10/10 batches | ms/batch 17.05 | loss 131.77 |
[10:18:31-081020]  | epoch 4 | loss 131.7674 | total time: 0.17s |
[10:18:31-081020]  | epoch 4 | ndcg@100 0.240 (0.0080) |
[10:18:31-081020]  | epoch 5 | 10/10 batches | ms/batch 15.79 | loss 132.81 |
[10:18:31-081020]  | epoch 5 | loss 132.8058 | total time: 0.16s |
[10:18:31-081020]  | epoch 5 | ndcg@100 0.256 (

### Evaluation

In [ ]:
from rectorch.evaluation import evaluate
results = evaluate(vae, sampler, ["ndcg@100", "recall@100", "ndcg@20", "recall@20"])

In [ ]:
from rectorch.utils import collect_results
collect_results(results)

{'ndcg@100': (0.4134699640575229, 0.2090619683418369),
 'ndcg@20': (0.3231564329510504, 0.22088148824524897),
 'recall@100': (0.6463630794000064, 0.2726948893343989),
 'recall@20': (0.38334497722392846, 0.26767216600690824)}

## Simulation of a "federated" scenario

In [9]:
sampler = DataSampler(dataset, mode="train", batch_size=1)
vae = MultVAE(dec_dims=[200,600,dataset.n_items],
              enc_dims=None,
              dropout=0.5,
              beta=.2,
              anneal_steps=100000,
              opt_conf={"name":"adam", "lr":0.0001},
              device="cuda",
              trainer=None)

In [ ]:
vae.train(sampler, valid_metric="ndcg@100")

## Federated VAE

In [10]:
from rectorch.samplers import DataSampler
import numpy as np

class Federated_Sampler(DataSampler):
    def __init__(self,
                 data,
                 mode="train",
                 batch_size=64,
                 seed=98765):
        super(Federated_Sampler, self).__init__(data, mode, batch_size, False)
        self.idxlist = list(range(self.sparse_data_tr.shape[0]))
        np.random.seed(seed)

    def __len__(self):
        return int(np.ceil(self.sparse_data_tr.shape[0] / self.batch_size))

    def _set_mode(self, mode="train", batch_size=None):
        super()._set_mode(mode, batch_size)
        self.idxlist = list(range(self.sparse_data_tr.shape[0]))

    def __iter__(self):
        if self.mode == "train":
            n = self.sparse_data_tr.shape[0]
            for _, start_idx in enumerate(range(0, n, self.batch_size)):
                np.random.shuffle(self.idxlist)
                data_tr = self.sparse_data_tr[self.idxlist[:self.batch_size]]
                yield torch.FloatTensor(data_tr.toarray()), None
        else:
            yield from super().__iter__()

In [11]:
from rectorch.models.nn.multvae import MultVAE_net
from rectorch.evaluation import evaluate
from rectorch.validation import ValidFunc
import copy
import torch
import torch.nn.functional as F
from torch.optim import Adam
from tqdm.notebook import tqdm

class FedMultVAE():
    def __init__(self):
        torch.manual_seed(98765)
        self.network = MultVAE_net(dec_dims=[200,600,dataset.n_items],
                       enc_dims=None,
                       dropout=0.5).to(torch.device("cuda"))
        self.device = torch.device("cuda")
        self.optimizer = Adam(self.network.parameters(), lr=0.001)

    def loss_function(self, recon_x, x, mu, logvar, beta=1.0):
        BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
        KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
        return BCE + beta * KLD

    def gradient_step(self, network, data_tensor):
        network.train()
        recon_batch, mu, var = network(data_tensor)
        loss = self.loss_function(recon_batch, data_tensor, mu, var, beta=.2)
        loss.backward()

    def sum_grad(self, net1, net2):
        for pA, pB in zip(net1.parameters(), net2.parameters()):
            pB.grad = pA.grad + (pB.grad if pB.grad is not None else 0)

    def train_epoch(self, epoch, data_sampler, valid_sampler):
        for data, _ in tqdm(data_sampler):
            self.optimizer.zero_grad()
            data_tensor = data.view(data.shape[0], -1).to(self.device)
            for u in range(len(data)):
                net_clone = copy.deepcopy(self.network)
                self.gradient_step(net_clone, data_tensor[u:u+1])
                self.sum_grad(net_clone, self.network)
            
            for p in self.network.parameters():
                p.grad /= len(data_sampler.idxlist)#len(data)

            self.optimizer.step()

            recon_batch, mu, var = self.network(data_tensor)
            loss = self.loss_function(recon_batch, data_tensor, mu, var, beta=.2)
            print("Loss: ", loss.item())
        
        return ValidFunc(evaluate)(self, valid_sampler, "ndcg@100")

    
    def train(self, data_sampler, valid_sampler, num_epochs=100):
        try:
            for e in range(num_epochs):
                print("Epoch %d" %(e+1))
                valid_res = self.train_epoch(e+1, data_sampler, valid_sampler)
                print("ndcg@100:", np.mean(valid_res))
        except KeyboardInterrupt:
            print("Interrupted!")

    def predict(self, x, remove_train=True):
        self.network.eval()
        with torch.no_grad():
            x_tensor = x.to(self.device)
            recon_x, mu, logvar = self.network(x_tensor)
            if remove_train:
                recon_x[torch.nonzero(x_tensor, as_tuple=True)] = -np.inf
            return recon_x, mu, logvar

In [ ]:
fed_vae = FedMultVAE()

fed_sampler = Federated_Sampler(dataset, batch_size=250)
np.random.seed(98765)
val_sampler = DataSampler(dataset, mode="valid", batch_size=250)

fed_vae.train(fed_sampler, val_sampler)

Epoch 1


Loss:  212.92596435546875
Loss:  184.6147003173828
Loss:  164.58154296875
Loss:  139.02503967285156
Loss:  157.7643280029297
Loss:  132.7198028564453
Loss:  144.79931640625
Loss:  188.6287078857422
Loss:  110.61787414550781
Loss:  155.79898071289062
Loss:  140.06591796875
Loss:  123.80813598632812
Loss:  162.65719604492188
Loss:  148.65811157226562
Loss:  157.99761962890625
Loss:  132.13685607910156
Loss:  115.03128051757812
Loss:  126.86376190185547
Loss:  140.7070770263672

ndcg@100: 0.2309752871487686
Epoch 2


Loss:  102.69071197509766
Loss:  136.58599853515625
Loss:  113.55171203613281
Loss:  111.09638214111328
Loss:  151.35867309570312
Loss:  99.89459228515625
Loss:  118.64105987548828
Loss:  127.5521469116211
Loss:  150.35147094726562
Loss:  151.20529174804688
Loss:  129.30355834960938
Loss:  121.16956329345703
Loss:  129.12742614746094
Loss:  128.92868041992188
Loss:  97.21497344970703
Loss:  114.91581726074219
Loss:  121.2171630859375
Loss:  127.08772277832031
Loss:  146.73013305664062

ndcg@100: 0.23319941687563772
Epoch 3


Loss:  165.3272247314453
Loss:  123.50934600830078
Loss:  175.91334533691406
Loss:  191.20323181152344
Loss:  183.02886962890625
Loss:  115.85818481445312
Loss:  157.12734985351562
Loss:  143.7069854736328
Loss:  161.35670471191406
Loss:  108.43341064453125
Loss:  133.18544006347656
Loss:  122.93038940429688
Loss:  121.51567077636719
Loss:  137.22283935546875
Loss:  92.24870300292969
Loss:  124.60784912109375
Loss:  103.15887451171875
Loss:  138.77957153320312
Loss:  132.56707763671875

ndcg@100: 0.2575923323525785
Epoch 4


Loss:  113.28900909423828
Loss:  101.57653045654297
Loss:  126.50460052490234
Loss:  156.20762634277344
Loss:  125.61925506591797
Loss:  139.8299560546875
Loss:  114.7417984008789
Loss:  122.7039794921875
Loss:  147.1260528564453
Loss:  165.3851776123047
Loss:  87.27008819580078
Loss:  136.62640380859375
Loss:  157.089599609375
Loss:  124.19992065429688
Loss:  114.18668365478516
Loss:  131.1461181640625
Loss:  113.68267059326172
Loss:  130.7022247314453
Loss:  137.6138458251953

ndcg@100: 0.27563528646870755
Epoch 5


...

Loss:  90.49571990966797
Loss:  104.41748046875
Loss:  88.94168853759766
Loss:  98.01512908935547
Loss:  103.0393295288086
Loss:  119.6401596069336
Loss:  115.81197357177734
Loss:  88.03919982910156
Loss:  103.66230773925781
Loss:  89.71526336669922
Loss:  101.32383728027344
Loss:  127.50253295898438
Loss:  87.39964294433594
Loss:  121.20584106445312
Loss:  105.24258422851562
Loss:  85.78546905517578
Loss:  97.83035278320312
Loss:  111.2845687866211
Loss:  94.89659881591797

ndcg@100: 0.4517222454212966
Epoch 35


Loss:  108.34952545166016
Loss:  108.9822998046875
Loss:  128.62313842773438
Loss:  125.69373321533203
Loss:  135.4807891845703
Loss:  136.48919677734375
Loss:  76.85009002685547
Loss:  132.31930541992188
Loss:  107.8752670288086
Loss:  89.35868835449219
Loss:  108.7494888305664
Loss:  107.76420593261719
Loss:  86.87100219726562
Loss:  132.32618713378906
Loss:  105.10675048828125
Loss:  93.84180450439453
Loss:  135.24717712402344
Loss:  76.95374298095703
Loss:  101.68498992919922

ndcg@100: 0.4507218355785917
Epoch 36


Loss:  107.73075866699219
Loss:  134.86419677734375
Loss:  125.48675537109375
Loss:  99.30525207519531
Loss:  98.5064926147461
Loss:  94.04337310791016
Loss:  94.8271713256836
Loss:  118.4834213256836
Loss:  106.56262969970703
Loss:  109.92304229736328
Loss:  88.16014099121094
Loss:  123.39315795898438
Loss:  107.94466400146484
Loss:  105.49240112304688
Loss:  109.77674865722656
Loss:  73.80953979492188
Loss:  97.75518798828125
Loss:  90.6556167602539
Loss:  94.36788940429688

ndcg@100: 0.4638026953492167
Epoch 37


Loss:  140.56039428710938
Loss:  87.97306060791016
Loss:  95.18136596679688
Loss:  99.2139663696289
Loss:  119.6434555053711
Loss:  85.83385467529297
Loss:  126.29850006103516
Loss:  105.9578857421875
Loss:  71.16301727294922
Loss:  111.70323944091797
Loss:  81.32087707519531
Loss:  101.61630249023438
Loss:  152.5348358154297
Loss:  55.704071044921875
Loss:  97.34774017333984
Loss:  132.30270385742188
Loss:  104.43248748779297
Loss:  91.33336639404297
Loss:  113.04237365722656

ndcg@100: 0.45636124369534126
Epoch 38


Loss:  84.77003479003906
Loss:  72.38521575927734
Loss:  95.65284729003906
Loss:  100.56072998046875
Loss:  78.7892837524414
Loss:  111.42992401123047
Loss:  103.83551025390625
Loss:  150.03009033203125
Loss:  118.50779724121094
Loss:  103.84943389892578
Loss:  57.23381805419922


In [ ]:
def compare(model1, model2):
    for p1, p2 in zip(model1.parameters(), model2.parameters()):
        if p1.data.ne(p2.data).sum() > 0:
            return False
    return True

In [91]:
from rectorch.models.nn.multvae import MultVAE_net
from rectorch.evaluation import evaluate
from rectorch.validation import ValidFunc
import copy
import torch
import torch.nn.functional as F
from torch.optim import Adam
from tqdm.notebook import tqdm

class FedMomMultVAE():
    def __init__(self, mom_beta=0.9):
        torch.manual_seed(98765)
        self.network = MultVAE_net(dec_dims=[200,600,dataset.n_items],
                       enc_dims=None,
                       dropout=0.5).to(torch.device("cuda"))
        self.device = torch.device("cuda")
        self.mom_beta = mom_beta
        self.optimizer = Adam(self.network.parameters(), lr=0.001)
    
    def exp_discount(self, bmin, bstart, d, x):
        return bmin + (bstart - bmin)*(1-d)**x

    def loss_function(self, recon_x, x, mu, logvar, beta=1.0):
        BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
        KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
        return BCE + beta * KLD

    def gradient_step(self, network, data_tensor):
        network.train()
        recon_batch, mu, var = network(data_tensor)
        loss = self.loss_function(recon_batch, data_tensor, mu, var, beta=.2)
        loss.backward()
    
    def momentum(self, w, v, beta=0.9):
        with torch.no_grad():
            for lw, lv in zip(w.enc_layers, v.enc_layers):
                lw.weight += beta * (lw.weight - lv.weight)

            for lw, lv in zip(w.dec_layers, v.dec_layers):
                lw.weight += beta * (lw.weight - lv.weight)

    def sum_grad(self, net1, net2):
        for pA, pB in zip(net1.parameters(), net2.parameters()):
            pB.grad = pA.grad + (pB.grad if pB.grad is not None else 0)

    def train_epoch(self, epoch, data_sampler, valid_sampler):
        for data, _ in tqdm(data_sampler):
            self.optimizer.zero_grad()
            v_t = copy.deepcopy(self.network)
            data_tensor = data.view(data.shape[0], -1).to(self.device)
            for u in range(len(data)):
                net_clone = copy.deepcopy(self.network)
                self.gradient_step(net_clone, data_tensor[u:u+1])
                self.sum_grad(net_clone, self.network)
            
            for p in self.network.parameters():
                p.grad /= len(data_sampler.idxlist)
            
            self.optimizer.step()
            v_t_prev = copy.deepcopy(self.network)
            mom = self.exp_discount(0., self.mom_beta, .1, epoch-1)
            #self.momentum(self.network, v_t, self.mom_beta)
            self.momentum(self.network, v_t, mom)
            v_t = v_t_prev

            recon_batch, mu, var = self.network(data_tensor)
            loss = self.loss_function(recon_batch, data_tensor, mu, var, beta=0.2)
            print("Loss: ", loss.item())
        
        return ValidFunc(evaluate)(self, valid_sampler, "ndcg@100")

    
    def train(self, data_sampler, valid_sampler, num_epochs=100):
        try:
            for e in range(num_epochs):
                print("Epoch %d" %(e+1))
                valid_res = self.train_epoch(e+1, data_sampler, valid_sampler)
                print("ndcg@100:", np.mean(valid_res))
        except KeyboardInterrupt:
            print("Interrupted!")

    def predict(self, x, remove_train=True):
        self.network.eval()
        with torch.no_grad():
            x_tensor = x.to(self.device)
            recon_x, mu, logvar = self.network(x_tensor)
            if remove_train:
                recon_x[torch.nonzero(x_tensor, as_tuple=True)] = -np.inf
            return recon_x, mu, logvar

In [92]:
#momentum
fed_vae = FedMomMultVAE(5.)

fed_sampler = Federated_Sampler(dataset, batch_size=250)
np.random.seed(98765)
val_sampler = DataSampler(dataset, mode="valid", batch_size=250)

fed_vae.train(fed_sampler, val_sampler)

Epoch 1


Loss:  788.0032958984375
Loss:  739.6207885742188
Loss:  972.0624389648438
Loss:  763.69482421875
Loss:  858.8525390625
Loss:  822.114013671875
Loss:  736.2301025390625
Loss:  745.2979736328125
Loss:  775.4385375976562
Loss:  631.00341796875
Loss:  758.2022094726562
Loss:  739.181884765625
Loss:  679.1088256835938
Loss:  682.8812255859375
Loss:  742.6539916992188
Loss:  655.8839721679688
Loss:  752.1309204101562
Loss:  749.4722290039062
Loss:  741.3133544921875
Loss:  781.84765625
Loss:  772.6560668945312
Loss:  694.41064453125
Loss:  677.486572265625

ndcg@100: 0.1903475034403344
Epoch 2


Loss:  722.1847534179688
Loss:  716.078857421875
Loss:  778.4502563476562
Loss:  719.5472412109375
Loss:  666.5320434570312
Loss:  661.7655029296875
Loss:  704.660888671875
Loss:  659.631103515625
Loss:  655.0825805664062
Loss:  707.7874755859375
Loss:  615.0848999023438
Loss:  746.800537109375
Loss:  617.8668823242188
Loss:  654.5107421875
Loss:  736.3836059570312
Loss:  678.6005249023438
Loss:  685.2532348632812
Loss:  729.8099975585938
Loss:  777.666748046875
Loss:  657.2597045898438
Loss:  600.6333618164062
Loss:  680.5123901367188
Loss:  699.587158203125

ndcg@100: 0.25895714796572183
Epoch 3


Loss:  620.2384643554688
Loss:  776.5328979492188
Loss:  630.9978637695312
Loss:  679.2003784179688
Loss:  642.6790771484375
Loss:  661.6810913085938
Loss:  710.2227172851562
Loss:  594.838134765625
Loss:  601.6121215820312
Loss:  661.4834594726562
Loss:  643.3515625
Loss:  660.3883666992188
Loss:  638.112060546875
Loss:  628.5928955078125
Loss:  702.2096557617188
Loss:  611.2900390625
Loss:  663.8499145507812
Loss:  643.6024169921875
Loss:  701.8316650390625
Loss:  750.5160522460938
Loss:  565.3655395507812
Loss:  717.802001953125
Loss:  630.0211791992188

ndcg@100: 0.3062413031862181
Epoch 4


Loss:  701.3826293945312
Loss:  687.3838500976562
Loss:  689.2728271484375
Loss:  647.851806640625
Loss:  680.953125
Loss:  623.2139892578125
Loss:  766.42138671875
Loss:  698.8984375
Loss:  680.539306640625
Loss:  636.4684448242188
Loss:  742.0789794921875
Loss:  622.7456665039062
Loss:  688.8782958984375
Loss:  615.076904296875
Loss:  661.0143432617188
Loss:  659.3590698242188
Loss:  567.38427734375
Loss:  678.0028076171875
Loss:  584.055908203125
Loss:  553.1998291015625
Loss:  624.8551025390625
Loss:  656.7817993164062
Loss:  650.52978515625

ndcg@100: 0.32785274113156143
Epoch 5


...

Loss:  629.8995361328125
Loss:  607.593994140625
Loss:  596.8480834960938
Loss:  641.71337890625
Loss:  582.4614868164062
Loss:  696.891845703125
Loss:  549.9734497070312
Loss:  660.8681030273438
Loss:  597.9567260742188
Loss:  628.5714111328125
Loss:  629.0733032226562
Loss:  615.97314453125
Loss:  603.8565063476562
Loss:  517.089599609375
Loss:  642.46630859375
Loss:  575.3411254882812
Loss:  567.5923461914062
Loss:  662.7792358398438
Loss:  680.2918090820312
Loss:  625.3042602539062
Loss:  542.9193115234375
Loss:  551.5963134765625
Loss:  566.5922241210938

ndcg@100: 0.40253351359029715
Epoch 48


Loss:  612.9650268554688
Loss:  589.2388916015625
Loss:  643.0737915039062
Loss:  604.8859252929688
Loss:  569.424072265625
Loss:  695.8968505859375
Loss:  593.8911743164062
Loss:  570.6607055664062
Loss:  572.5652465820312
Loss:  618.500244140625
Loss:  595.5988159179688
Loss:  599.781005859375
Loss:  671.20556640625
Loss:  594.389892578125
Loss:  537.308349609375
Loss:  672.7681274414062
Loss:  597.4853515625
Loss:  576.6725463867188
Loss:  632.7030639648438
Loss:  569.7588500976562
Loss:  591.6320190429688
Loss:  550.3488159179688
Loss:  568.479736328125

ndcg@100: 0.41096033610692556
Epoch 49


Loss:  543.7794189453125
Loss:  577.2091674804688
Loss:  595.2254638671875
Loss:  573.3009643554688
Loss:  571.4492797851562
Loss:  580.8814086914062
Loss:  560.2239379882812
Loss:  577.239501953125
Loss:  566.7373657226562
Loss:  622.1580810546875
Loss:  642.8797607421875
Loss:  612.6843872070312
Loss:  573.44580078125
Loss:  596.1099243164062
Loss:  550.3853759765625
Loss:  544.3690795898438
Loss:  571.6117553710938
Loss:  621.6926879882812
Loss:  621.917236328125
Loss:  590.1703491210938
Loss:  627.7493286132812
Loss:  505.4104919433594
Loss:  561.0618896484375

ndcg@100: 0.40607967613895996
Epoch 50


Loss:  688.1841430664062
Loss:  584.0913696289062
Loss:  571.5393676757812
Loss:  581.3350219726562
Loss:  584.3131103515625
Loss:  614.9357299804688
Loss:  640.4811401367188
Loss:  652.7341918945312
Loss:  502.3864440917969
Loss:  570.0491333007812
Loss:  638.7574462890625
Loss:  581.57080078125
Loss:  562.1925659179688
Loss:  563.5548095703125
Loss:  631.6860961914062
Loss:  652.46337890625
Loss:  541.3519897460938
Loss:  644.727294921875
Loss:  542.5693969726562
Loss:  630.171630859375
Loss:  625.6021728515625
Loss:  632.7169189453125
Loss:  567.6015625

ndcg@100: 0.40477080901896273
Epoch 51


Loss:  525.3030395507812
Loss:  736.5284423828125
Interrupted!
